In [1]:
# Import relevant packages
from math import log
from __future__ import print_function
import pandas as pd
import implicit
import numpy as np
import scipy
import random
from operator import itemgetter
from tqdm import tqdm_notebook as tqdm
import seaborn as sns
from matplotlib import pyplot as plt
from pylab import savefig
import time 

# Import main dataset
df = pd.read_csv('lastfm_9000_users.csv', na_filter=False)
df = df.drop(['Unnamed: 0'], axis=1)

## Data Preparation|

create_sparse_matrix and calculate_sparsity are functions that allow us to set up the dataset in sparse matrix form.

In [2]:
# Create sparse matrix from dataframe object
def create_sparse_matrix(data, user_user = True):
    """
    Creates sparse matrix (csr_matrix) out of pandas dataframe.
    
    Parameters: 
    - data: Dataframe of user/artist data
    - user_user: determines whether output will be for user-to-user or item-to-item collaborative filtering
                 if user_user is True, then rows will be items and columns will be users
    
    Returns: 
    - plays_sparse: a sparse csr_matrix
    
    """
    print("Creating sparse matrix...")
    #grab unique users/artist IDS
    users = list(np.sort(data.user_id.unique()))
    artists = list(data.artist_mbid.unique())
    plays = list(data.plays)

    # user-user set-up
    if (user_user == True):
        rows = data.user_id.astype('category', categories=users).cat.codes
        cols = data.artist_mbid.astype('category', categories=artists).cat.codes
        plays_sparse = scipy.sparse.csr_matrix((plays, (rows, cols)), shape=(len(users),len(artists)))

    #item-item set-up
    else:    
        rows = data.artist_mbid.astype('category', categories=artists).cat.codes
        cols = data.user_id.astype('category', categories=users).cat.codes
        plays_sparse = scipy.sparse.csr_matrix((plays, (rows, cols)), shape=(len(artists),len(users)))
        
    return plays_sparse

In [3]:
# Calculate sparsity of mnatrix
def calculate_sparsity(M):
    """
    Computes sparsity of matrix
    
    params:
        M: matrix to be computed
    """
    matrix_size = float(M.shape[0]*M.shape[1]) # Number of possible interactions in the matrix
    num_plays = len(M.nonzero()[0]) # Number of items interacted with
    sparsity = 100*(1 - float(num_plays/matrix_size))
    return sparsity

## Split Train Test

These two functions are used to split the dataset into test/train/tune in various ways. 
make_train_all_user_pairs splits the data into a test and training set with a proportion taken over all users, while split_train_test_per_user makes sure that a certain proportion of each user is held off for the test set. 

split_train_test_per_user also allows for the option of k-fold cross validation. 

In [ ]:
make_train_all_users_pairs

In [4]:
# Split train, test using all user-artist pairs
def make_train_all_user_pairs(data, test_pct):
    """
    params:
        data: data set in csr_matrix format
        test_pct: percentage to be test set
    """
    # Create copies of dataset for training and test data
    test = data.copy()
    train = data.copy()
    
    #alter train data, masking/holding-out random user-pair values for some users
    nonzero_idx = train.nonzero() #find indices in data where interaction exists
    
    
    nonzero_pairs = zip(nonzero_idx[0], nonzero_idx[1]) #create pairs of (user, item) index
    
    # Determine how many user-pair values we need to mask, according to test_pct
    random.seed(0) #for reproducibility
    num_samples = int(np.ceil(test_pct * len(nonzero_pairs)))
    
    #Sample random number of user-item pairs without replacement
    samples = random.sample(nonzero_pairs, num_samples) 
    print(type(samples))
    
    # Get user, item row and column indices
    user_idx = [index[0] for index in samples] 
    item_idx = [index[1] for index in samples] 
    
    # Mask the randomly chosen user-item pairs
    train[user_idx,item_idx] = 0 
    # Remove zeros in sparse arrays that was made previously
    train.eliminate_zeros() 
    
    return train, test, list(set(user_idx)), samples #output unique list of user rows that were altered

In [5]:
# Split train, test by user only with interactions#, with test size=total/k
def split_train_test_per_user(data, k, interactions = 20,cross_valid= False):
    """
    Create train matrix with masked values and dictionary of test values 
    
    Parameters:
    - data: csr_matrix, assuming matrix is user-user (users as rows, columns as items)
    - test_pct: percentage of items to mask per user
    
    Output:
    - train: masked matrix
    - test: list of tuples of held out data ((user_idx, item_idx), plays)
    """
    random.seed(0) #for reproducibility
    
    train = data.copy() #transpose to make procedure easier/more intuitive
    
    test = dict() #dict to keep track of masked user-item values
    
    user_count = 0
    test_list=[]
    train_list=[]
    if cross_valid==True: #initialize
        for i in range(k):
            test_list.append(dict())
            train_list.append(train)
    
    for user_idx in tqdm(range(train.get_shape()[0])):

        # Get indices of interactions of this user
        nonzero_idx = train[user_idx].nonzero()

        # Only hold out users that have enough data (greater than interactions #)
        if nonzero_idx[1].shape[0] >= interactions:
            user_count += 1
            # Create list of tuples: interaction index (row, col) with the number of plays
            nonzero_pairs = [((user_idx, item_idx), train[user_idx,item_idx]) for item_idx in nonzero_idx[1]]

            # Sort tuples by descending value
            nonzero_sorted = sorted(nonzero_pairs, key = itemgetter(1), reverse = True)

            # Get top interaction # values, then sample test_pct% randomly from subset
            top_values = nonzero_sorted[0:interactions]

            # Sample random number of item_indexes without replacement
            num_samples = int(np.floor(interactions/float(k)))
            if (cross_valid==False): 
                samples = random.sample(top_values, num_samples) 

                # Append user_idx, item_
                test[user_idx] = [pair[0][1] for pair in samples]

                # Mask the randomly chosen items of this user
                for pair in samples:
                    train[pair[0][0], pair[0][1]] = 0
            
            # Cross Validation Step
            else:
                for i in range(k):
                    train = train_list[i]
                    k_test=test_list[i]
                    random.shuffle(top_values) 
                    samples=top_values[0:num_samples]
                    top_values=top_values[num_samples:]
                    # Append user_idx, item_
                    k_test[user_idx] = [pair[0][1] for pair in samples]
                    test_list[i]=k_test #update test
                    # Mask the randomly chosen items of this user
                    for pair in samples:
                        train[pair[0][0], pair[0][1]] = 0
                    train.eliminate_zeros()
                    # Update train
                    train_list[i]=train 
    if (cross_valid==False):
        # Convert matrix back to initial shape
        return train.T.tocsr(), test, user_count
    else:
        for i in range(k):
            train_list[i]=train_list[i].T.tocsr()
        # Convert matrix back to initial shape
        return train_list, test_list, user_count


In [6]:
# Calculate how many interactions are masked compared to previous dataset
def pct_masked(original, altered):
    altered_n = altered.nonzero()[0].shape[0]
    original_n = original.nonzero()[0].shape[0]
    return (original_n - altered_n)/float(altered_n)

### Baseline Implementation

Below is how we generated our baseline recommendations (taking the most popular artists across the entire dataset and recommending them to everyone)

In [93]:
class Baseline():
    """
    Baseline model. Take most popular artist across entire dataset. 
    """
    def __init__(self, n_recs):
        self.n_recs = n_recs
    
    def fit(self, user_items):
        print("Fitting baseline...")
        plays = user_items.toarray()
        total_plays = np.sum(plays, axis = 1)
        idx = (-total_plays).argsort()[:self.n_recs]
        self.idx = idx
    
    def predict(self, X=None):
        return self.idx


## Evaluation/Metrics

The following are the functions we wrote to determine the NDCG/recall of the baseline, ALS, and KNN recommendations. 

auto_tune_parameter is a function written to determine the best hyperparameters to use for a given model. 

In [151]:
def zeros_list(n):
    listofzeros = [0] * n
    return listofzeros

def dcg_at_k(scores):
    assert scores
    return scores[0] + sum(sc / log(ind, 2) for sc, ind in zip(scores[1:], range(2, len(scores)+1)))

def ndcg_at_k(rec_items, holdout_items):
    """
    rec_items: recommended k items from model
    heldout_items: held out items
    """
    assert len(rec_items) == len(holdout_items)
    idcg = dcg_at_k(sorted(holdout_items, reverse=True))
    ndcg = (dcg_at_k(rec_items) / idcg) if idcg > 0.0 else 0.0
    
    return ndcg

Main Evaluation, Include Recall and Precision

In [200]:
# Used to evaluate model
def evaluate(model, model_name, test, M_train, n_rec = 20):
    """
    Calculate evaluation metrics (precision@k, recall@k, NDCG@k)
    
    parameters:
    - model: fitted implicit model that will perform recommendations
    - model_name: name of package for switch case
    - test: list containing tuples that are heldout for each user
    - M_train: csr_matrix of user-item pairs, used in fit (user by item)
    - n_rec: how many recommendations the system outputs
    
    returns:
    - two numpy arrays containing precision and recall
    """
    print('Evaluating model...')
    
    #to store results
    ndcg = []
    user_n = 0.0
    test_n = 0.0 #keep track of number of heldout items
    tp = 0.0 #true positive 
    
    for user, holdout_items in tqdm(test.items()):
        
        user_n += 1
        test_n += len(holdout_items)
        
        #for NDCG
        predicted_items = zeros_list(n_rec)
        true_items = zeros_list(n_rec)
        
        #get recommended items from models for user
        if model_name == "baseline": 
            rec_items = model.predict()

        elif model_name == "implicit":
            rec_items = model.recommend(user, M_train.T.tocsr(), N=n_rec, filter_already_liked_items=True) #returns (item_id, score)
            rec_items = [pair[0] for pair in rec_items] #get only item_
        
        elif model_name == "lightfm":
            #sort scores, find top recommended items
            rec_items = (-model.predict([user],range(1,M_train.T.tocsr().shape[1]))).argsort()[:n_rec]
        
        else:
            raise ValueError("Model may not be supported. Check if model name is correct.")
        
        #if np array change to list
        if isinstance(rec_items, np.ndarray):
            rec_items = rec_items.tolist()

        #index for holdout items
        i = 0
        for item in holdout_items:
            value = M_train[user,item] #get plays value of this holdout item
            true_items[i] = value
            i += 1

            if item in rec_items:
                tp += 1
                predicted_items[rec_items.index(item)] = value #get plays value of true positive

        ndcg.append(ndcg_at_k(predicted_items, true_items))

    recall = tp/test_n
    precision = tp/(n_rec * user_n)
    avg_ndcg = np.mean(ndcg)

    return precision, recall, avg_ndcg

In [10]:
"""
Function that identifies optimal parameter values given relevant models and arrays of parameters

input: 
    - k: # of folds within the training set (split into tuning sets)
    - interactions: size of recommendation list
    - model: model that is being optimized   
    - data: sparse user-item matrix
    - param1: list of values to try for hyperparameter 1.
    - param2: list of values to try for hyperparameter 2. 
    - param3: list of values to try for hyperparameter 3.
    NOTE: if using KNN, there are 3 parameters. param2 is the integer used for k1; 
    only k and b are tested only param1 and param3 should be lists.
    If using ALS, there are 2 parameters. both should be lists. 
    
output:
    - max_ndcg_list: a list of k tuples, one for each fold. 
        each tuple is in the form (max_ndcg,max_first_param,max_second_param,max_recall)
        which records the best ndcg, and the two params that achieved it, 
        and the max_recall achieved (which may be from different param values).
    - heatmap_list: a list of k heatmaps of the NDCG values for the two tested 
        parameters (one heatmap per fold). Useful for visualizations
"""
def auto_tune_parameter(k,interactions,model,data,param1,param2,param3=None):
    # Train model
    # Create list of MAX NDCG and Recall depending on # params
    max_ndcg_list=[] #will end up being length k list of tuples of best param values
    heatmap_list=[]
    if param3==None: #for ALS
        num_param=2
        first_param=param1
        second_param=param2
    else: #for KNN
        num_param=3
        first_param=param1
        second_param=param3
    train_and_tune,test,user_count=split_train_test_per_user(data, k+1, interactions,cross_valid= False)
    train_list, tune_list, user_count = split_train_test_per_user(train_and_tune.T.tocsr(),k,int(np.ceil(((k-1)/k)*interactions)),cross_valid=True)
    #to be updated via max: to determine final params to use on test set 
    test_ndcg=0
    test_first_param=first_param[0]
    test_second_param=second_param[0]
    #create recall/NDCG matrix storing for each combination of 
    for fold in range(k): #For each fold; there are k-1 folds within train_and_tune
        ndcg_heatmap=[[0 for x in range(len(second_param))] for y in range(len(first_param))]
        print(ndcg_heatmap)
        train=train_list[fold]
        tune=tune_list[fold]
        max_first_param=first_param[0] #initialize best value of first_param for this fold
        max_second_param=second_param[0] #initialize best value of second_param for this fold
        max_recall=0
        max_ndcg=0
        value1_index=0 #index for heatmap
        print("Fitting fold number...",fold)
        for value1 in first_param:
            value2_index=0
            for value2 in second_param:
                if num_param==2:
                    print("Trying ",(value1,value2))
                    usemodel=model(value1,value2)
                if num_param==3:
                    print("Trying ",(value1,param2,value2))
                    usemodel=model(value1,param2,value2)
                usemodel.fit(train, show_progress=False)
                recall,ndcg = evaluate(usemodel,tune,data)
                print(value1_index,value2_index)
                ndcg_heatmap[value1_index][value2_index]=ndcg #update heatmap 
                #recall_list.append(recall)
                #update maximum values
                max_recall=max(max_recall,recall)
                if ndcg>max_ndcg: 
                    max_ndcg=ndcg
                    max_first_param=value1
                    max_second_param=value2
                value2_index=value2_index+1
            value1_index=value1_index+1    
        max_ndcg_list.append([max_ndcg,max_first_param,max_second_param,max_recall])
        if max_ndcg>test_ndcg:
            print("Fold ",fold," beat the record for ncdg!")
            print("New best ndcg is ",max_ndcg)
            print("New best params are ",(max_first_param,max_second_param))
            test_ndcg=max_ndcg
            test_first_param=max_first_param
            test_second_param=max_second_param
        heatmap_list.append(ndcg_heatmap)
        print("end of fold---------------------------")

    #Now, test_first_param and test_second_param should be optimized
    if num_param==2:
        usemodel=model(test_first_param,test_second_param)
    if num_param==3:
        usemodel=model(test_first_param,param2,test_second_param)
    usemodel.fit(train_and_tune,show_progress=True)
    final_recall,final_ndcg = evaluate(usemodel,test,data)
    
    print("The recall on the test set is ", final_recall,", after hyperparameter optimization")
    print("The ndcg on the test set is ",final_ndcg,", after hyperparameter optimization")
    
    return max_ndcg_list,heatmap_list 

# Main Script

#### 1. Setup: Create sparse matrix

In [187]:
#create sparse matrix
plays_sparse = create_sparse_matrix(df).astype('float')
print('Matrix Sparsity:', calculate_sparsity(plays_sparse))

Creating sparse matrix...


/Users/TimGimi/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/Users/TimGimi/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:24: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead


Matrix Sparsity: 99.8965986346


In [188]:
#user by item matrix
plays_sparse

<9000x47102 sparse matrix of type '<type 'numpy.float64'>'
	with 438337 stored elements in Compressed Sparse Row format>

In [182]:
# Split data into training and test sets
train, test, user_count = split_train_test_per_user(plays_sparse, 3, 10)

In [190]:
print("Percentage of original data masked:", pct_masked(plays_sparse, train.T.tocsr()))
print("Users masked:", user_count)

Percentage of original data masked: 0.0654841916689
Users masked: 8980


#### Model-Based (ALS)

Here we run the model-based ALS Matrix Factorization and display the results. 

In [204]:
model = implicit.bpr.BayesianPersonalizedRanking(50)

# Train model
print("Fitting model...")
model.fit(train, show_progress=True)

# Compute and output recall and NDCG
precision, recall, ndcg = evaluate(model, 'implicit', test, plays_sparse)
print("Recall:",recall*100,'%')
print("Precision:",precision*100,'%')
print("Average NDCG:",ndcg*100,'%')

  0%|          | 0/100 [00:00<?, ?it/s]

Fitting model...


100%|██████████| 100/100 [00:28<00:00,  3.47it/s, skipped=2.93%, correct=94.14%]

Evaluating model...


Recall: 22.293986637 %
Precision: 3.34409799555 %
Average NDCG: 12.58182176339819 %


#### K-Nearest Neighbors (Item-Based)

Here we run the KNN and display the results.

In [195]:
model_knn = implicit.nearest_neighbours.BM25Recommender()

# Train Model
print("Fitting model...")
model_knn.fit(train, show_progress=True)

# Compute and output recall and NDCG
precision, recall, ndcg = evaluate(model_knn, 'implicit', test, plays_sparse)
print("Recall:",recall*100,'%')
print("Precision:",precision*100,'%')
print("Average NDCG:",ndcg*100,'%')

  2%|▏         | 805/47102 [00:00<00:05, 8044.62it/s]

Fitting model...


100%|██████████| 47102/47102 [00:00<00:00, 78234.35it/s]


Evaluating model...


Recall: 3.12917594655 %
Precision: 0.469376391982 %
Average NDCG: 1.6164508248527942 %


#### Baseline

Here we run baseline and display the results.

In [201]:
baseline = Baseline(n_recs = 20)
baseline.fit(train)

# Compute and output recall and NDCG
precision, recall,ndcg = evaluate(baseline, 'baseline', test, plays_sparse)
print("Recall:",recall*100,'%')
print("Precision:",precision*100,'%')
print("Average NDCG per User:",ndcg*100,'%')

Fitting baseline...
Evaluating model...


Recall: 7.61692650334 %
Precision: 1.1425389755 %
Average NDCG per User: 4.533232513791051 %


# Cross Validation and Parameter Tuning (k-fold)

In this section, we use k-fold cross validation to tune hyperparameters and evaluate our models.

### Splitting into test and training sets

In [16]:
# Cross Validation test
k=5
train_list, test_list, user_count = split_train_test_per_user(plays_sparse,k,20,cross_valid=True)

## Evaluate and tune ALS

Tuning two parameters: number of latent factors and the regularization factor. 

For latent factors, we try values [10,20,30,40,50,60]

For regularization factors, we try [.01,.03,.05,.07]

In [ ]:
start = time.time()
model=implicit.als.AlternatingLeastSquares
ndcg_list,heatmap_list=auto_tune_parameter(4,20,model,plays_sparse,[10,20,30,40,50,60],[.01,.03,.05,.07],param3=None)
stop = time.time()
total = stop-start

In [ ]:
# Plot heatmap of parameter tuning results
sns.set_style("whitegrid")
sns.heatmap(heatmap_list[2], 
            xticklabels=['0.01','0.03','0.05','0.07'], 
            yticklabels=['10','20','30','40','50','60'], 
            cbar_kws={'label':'NDCG Score'})
plt.ylabel("Number of Latent Factors")
plt.xlabel("Regularization Factor")
plt.title("ALS NDCG scores according to Model Parameters")
plt.show()

Here we analyze the scalability of ALS by looking at datasets with 9k, 20k, 60k, and 150k users:

In [ ]:
# The following block imports larger datasets for scaling tests, these expanded CSVs are not available in the repo
files9k = pd.read_csv('lastfm_9000_users.csv', na_filter=False)
files20k = pd.read_csv('lastfm_20k_users.csv', na_filter=False)
files60k = pd.read_csv('lastfm_60k_users.csv', na_filter=False)
files150k = pd.read_csv('lastfm_150k_users.csv', na_filter=False)
files40k = get_users(files150k, 40000)
files = [files9k, files20k, files40k, files60k]

In [ ]:
# Compute the recall and ndcg using optimal parameters for the ALS model on different dataset sizes
size = [9000, 20000, 40000, 60000]
ndcg_size = []
recall_size = []
for i in files:
    model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.01)

    #create sparse matrix
    plays_sparse = create_sparse_matrix(i).astype('float')
    print('Matrix Sparsity:', calculate_sparsity(plays_sparse))

    train, test, user_count = split_train_test_per_user(plays_sparse, 4, 20)

    # train model 
    print("Fitting model...")
    model.fit(train, show_progress=True)

    recall, ndcg = evaluate(model, test, plays_sparse)
    print("Recall:",recall*100,'%')
    print("Average NDCG:",ndcg*100,'%')
    recall_size.append(recall)
    ndcg_size.append(ndcg)

In [ ]:
# Plot scalability of ALS model
ndcg_size_df = pd.DataFrame({'N':size,
                       'NDCG': ndcg_size})
g = sns.pointplot(x='N', y='NDCG', data=ndcg_size_df)
plt.title('NDCG by Input Size for ALS')
plt.xlabel('Number Of Users')
plt.show()

Here we analyze the catalog coverage of the ALS model:

In [ ]:
# Calculate catalog coverage of ALS model with optimal parameters
model = implicit.als.AlternatingLeastSquares(factors=30, regularization=0.01)
model.fit(plays_sparse)
users = list(df.user_id.unique())
catalog = []
for i in range(0,len(users)):
    for x,y in model.recommend(i,plays_sparse.T.tocsr(), N=20, filter_already_liked_items=True):
        if x not in catalog:
            catalog.append(x)
print('Catalog Coverage is', len(catalog)/plays_sparse.shape[1])

## Evaluate and Tune KNN 

Tuning two parameters: K and B. 

For K, we try values [200,400,600,800,1000]

For K1, we stick to default value of 1.2

For B, we try [0,0.5,1]

In [ ]:
# Tune KNN to identify the best parameters
model=implicit.nearest_neighbours.BM25Recommender
max_ndcg_list, heatmap_list = auto_tune_parameter(4,20,model,plays_sparse,[200,400,600,800,1000],1.2,[0,0.5,1])

In [ ]:
# Plot heatmap of parameter tuning results
sns.heatmap(heatmap_list[1], 
            yticklabels=['1000', '800', '600', '400', '200'], 
            xticklabels=[0,0.5,1],
            cbar_kws={'label': 'NDCG Score'})
plt.xlabel('B')
plt.ylabel('Number of Neighbors')
plt.title('KNN Parameter Tuning With NDCG')
plt.show()

In [ ]:
# Compute the recall and ndcg using optimal parameters for the ALS model on different dataset sizes
size = [9000, 20000, 40000, 60000]
ndcg_size_knn = []
recall_size_knn = []
for i in files:
    model = implicit.nearest_neighbours.BM25Recommender(K=400, K1=1.2, B=0)

    #create sparse matrix
    plays_sparse = create_sparse_matrix(i).astype('float')
    print('Matrix Sparsity:', calculate_sparsity(plays_sparse))

    train, test, user_count = split_train_test_per_user(plays_sparse, 4, 20)

    # train model 
    print("Fitting model...")
    model.fit(train, show_progress=True)

    recall, ndcg = evaluate(model, test, plays_sparse)
    print("Recall:",recall*100,'%')
    print("Average NDCG:",ndcg*100,'%')
    recall_size_knn.append(recall)
    ndcg_size_knn.append(ndcg)

Here we analyze the scalability of the KNN model:

In [ ]:
# Plot scalability of KNN model
ndcg_size_knn_df = pd.DataFrame({'N':size,
                       'NDCG': ndcg_size_knn})
g = sns.pointplot(x='N', y='NDCG', data=ndcg_size_knn_df)
plt.title('NDCG by Input Size for KNN')
plt.xlabel('Number Of Users')

Here we analyze the catalog coverage of the KNN model:

In [ ]:
# Calculate catalog coverage of KNN model with optimal parameters
model = implicit.nearest_neighbours.BM25Recommender(K=400, K1=1.2, B=0)
model.fit(plays_sparse)
users = list(df.user_id.unique())
catalog = []
for i in range(0,len(users)):
    for x,y in model.recommend(i,plays_sparse.T.tocsr(), N=20, filter_already_liked_items=True):
        if x not in catalog:
            catalog.append(x)
print('Catalog Coverage is', len(catalog)/plays_sparse.shape[1])

# Tune ALS

Below we will tune the hyperparameters of ALS from a given range of hyperparameters.

In [ ]:
model=implicit.als.AlternatingLeastSquares
ndcg_list,heatmap_list=auto_tune_parameter(4,20,model,plays_sparse,[10,20,30,40,50,60],[.01,.03,.05,.07],param3=None)

sns.set_style("whitegrid")
sns.heatmap(heatmap_list[2], xticklabels=['0.01','0.03','0.05','0.07'], yticklabels=['10','20','30','40','50','60'], cbar_kws={'label':'NDCG Score'})
plt.ylabel("Number of Latent Factors")
plt.xlabel("Regularization Factor")
plt.title("ALS NDCG scores according to Model Parameters")
plt.show()

# Input Size vs Training Time

Below you can run the code using a larger dataset to evaluate performance tiem compared with input tiem

In [ ]:
#helper function to grab needed users
def get_users(df, n):
    sample_userid = df["user_id"].unique()
    sample_userid = np.random.choice(sample_userid, size = n, replace = False)

    #grab rows with sample user id
    df_sample = df[df.user_id.isin(sample_userid)].reset_index(drop = True)

    return df_sample

#in order to run this analysis, you need to download the 150k data at: 
#https://www.dropbox.com/s/qd8rnlxsuq0rjll/last_fm_bigger_data.zip?dl=0

#read in large dataset
df_150 = pd.read_csv('lastfm_150k_users.csv', na_filter=False)
df_150 = df_150.drop(['Unnamed: 0'], axis=1)

# KNN 

In [ ]:
#plot KNN training time vs input size
size = [9000,20000,40000,60000,80000]
train_time = list()

#get train data for each input size
for users in size:
    df = get_users(df_150, users)
    plays_sparse = create_sparse_matrix(df)

    # K-Nearest Neighbors
    model = implicit.nearest_neighbours.BM25Recommender()
    
    start = time.time()
    # train model 
    print("Fitting model...")
    model.fit(plays_sparse, show_progress=True)
    stop = time.time()
    total = stop-start
    train_time.append(total)

#plot
df = pd.DataFrame({'n':size,'time':train_time})
sns.pointplot(x="n",y="time",data=df)
plt.title('Training Time by Input Size for KNN')
plt.xlabel('Number of Users')
plt.ylabel('Time (s)')

# ALS

In [ ]:
#plot ALS training time vs input size
#calculate training time for different input sizes for ALS
import sns
import time

size = [9000,20000,40000,60000,80000]
train_time = list()

#get train data for each input size
for users in size:
    df = get_users(df_150, users)
    plays_sparse = create_sparse_matrix(df)

    # K-Nearest Neighbors
    model = implicit.als.AlternatingLeastSquares(50)
    
    start = time.time()
    # train model 
    print("Fitting model...")
    model.fit(plays_sparse, show_progress=True)
    stop = time.time()
    total = stop-start
    train_time.append(total)

#plot
df = pd.DataFrame({'n':size,'time':train_time})
sns.pointplot(x="n",y="time",data=df)
plt.title('Training Time by Input Size for KNN')
plt.xlabel('Number of Users')
plt.ylabel('Time (s)')